In [1]:
# import duckdb
# import polars as pl
from operator import itemgetter
from ipyleaflet import Map, Marker
from ipywidgets import Layout, HTML

In [2]:
%reload_ext sql

Loading configurations from /home/andy/Works/Python/Flambe/pyproject.toml.

Settings changed:

Config,value
feedback,False
autopandas,False
autopolars,True
displaycon,False
displaylimit,1000


In [3]:
%sql duckdb:////data/duckdb/foursquare.duckdb

In [4]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

Success
bool


In [5]:
pike_lane_cottage = (51.69220544124235, -2.222896814346314)
home_lat, home_lon = pike_lane_cottage

In [6]:
%%sql 
pike_df << SELECT name, website, address, locality, region, postcode, country, 
                  latitude, longitude, point, 
                  ST_Distance(point, ST_Point({{home_lat}}, {{home_lon}})) AS distance
             FROM fs 
            WHERE postcode LIKE 'GL6 %'
            ORDER BY distance ASC

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
print(pike_df)

shape: (1_373, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ name      ┆ website   ┆ address   ┆ locality  ┆ … ┆ latitude  ┆ longitude ┆ point     ┆ distance │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ f64       ┆ f64       ┆ binary    ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Garling   ┆ http://ww ┆ Gigg      ┆ Nailswort ┆ … ┆ 51.692178 ┆ -2.222711 ┆ b"\x00\x0 ┆ 0.000188 │
│ Consultin ┆ w.roberto ┆ Mill, Old ┆ h         ┆   ┆           ┆           ┆ 0\x00\x00 ┆          │
│ g         ┆ piecollec ┆ Bristol   ┆           ┆   ┆           ┆           ┆ \x00\x00\ ┆          │
│           ┆ tio…      ┆ Rd Unit…  ┆           ┆   ┆           ┆           ┆ x00…      ┆          │
│ Mumford   ┆ null      ┆ Gigg Mill ┆ Nailswort ┆ … ┆ 51.692178 ┆ -2.222

In [8]:
print(pike_df.glimpse())

Rows: 1373
Columns: 11
$ name         <str> 'Garling Consulting', 'Mumford Specialist Vehicles', 'Trickster Productions', 'Stroud Building', 'The Wire Watcher Co', 'Danarm', 'Harley Musical Instruments', 'Passward Financial Planning', 'G E', 'IDig3Dprinting'
$ website      <str> 'http://www.robertopiecollection.com', None, None, None, 'http://www.wirewatcher.co.uk', 'http://www.danarm.com', None, None, 'http://www.geeltd.co.uk', 'http://www.idig3dprinting.co.uk'
$ address      <str> 'Gigg Mill, Old Bristol Rd Unit 6', 'Gigg Mill', 'Gigg Mill, Old Bristol Rd', 'Gigg Mill Old Bristol Rd', 'Gigg Mill, Old Bristol Rd', 'Gigg Mill, Old Bristol Rd Unit 1', 'Gigg Mill, Old Bristol Rd', 'Gigg Mill, Old Bristol Rd', 'Gigg Mill, Old Bristol Rd', None
$ locality     <str> 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth', 'Nailsworth'
$ region       <str> 'Gloucestershire', 'Gloucestershire', 'Gloucestershire', 'Gloucester

In [ ]:
map = Map(
    center=pike_lane_cottage,
    zoom=15,
    layout=Layout(width="100%", height="800px"),
)


def on_location_changed(event):
    print(event)


home = Marker(
    location=pike_lane_cottage,
    draggable=True,
    color="red",
)
home.observe(on_location_changed, "location")
map.add(home)

get_addr = itemgetter(
    "address",
    "locality",
    "region",
    "postcode",
    "country",
)

for place in pike_df.head(500).to_dicts():
    marker = Marker(
        location=(place["latitude"], place["longitude"]),
        title=place["name"],
        draggable=False,
    )

    name = place["name"]
    if place["website"]:
        name = f'<a href="{place["website"]}">{name}</a>'

    address_parts = [part for part in get_addr(place) if part is not None]
    address = "<br>".join(address_parts)

    popup = HTML()
    popup.value = f"<h3>{name}</h3><div>{address}</div>"
    marker.popup = popup
    map.add(marker)

display(map)

Map(center=[51.69220544124235, -2.222896814346314], controls=(ZoomControl(options=['position', 'zoom_in_text',…